## PAVICS Web Processing Services 
PAVICS allows access to a number of different WPS services via Birdhouse
* Each 'bird' groups a set of processing tools 


In [10]:
from owslib.wps import WebProcessingService
import requests
import xml.etree.ElementTree as ET


### One suite of WPS tools for netcdf files resides in 'Hummingbird'
For metadata use GetCapabilities

In [11]:
# Hummingbird WPS url
wps_url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/hummingbird/wps'
# connection
wps = WebProcessingService(url=wps_url)
# print wps title
wps.identification.title

'Hummingbird 0.5_dev'

###Print out info on available processes (from Hummingbird)

In [12]:
for process in wps.processes:
    print ('%s \t : %s \n' %(process.identifier, process.abstract))

ncdump 	 : Run ncdump to retrieve NetCDF header metadata. 

spotchecker 	 : Checks a single uploaded or remote dataset against a variety of compliance standards. The dataset is either in the NetCDF format or a remote OpenDAP resource. Available compliance standards are the Climate and Forecast conventions (CF) and project specific rules for CMIP6 and CORDEX. 

cchecker 	 : Runs the IOOS Compliance Checker tool to check datasets against compliance standards. Each compliance standard is executed by a Check Suite, which functions similar to a Python standard Unit Test. A Check Suite runs one or more checks against a dataset, returning a list of Results which are then aggregated into a summary. Development and maintenance for the compliance checker is done by the Integrated Ocean Observing System (IOOS). 

cfchecker 	 : The NetCDF Climate Forcast Conventions compliance checker by CEDA. This process allows you to run the compliance checker to check that the contents of a NetCDF file comply 

### PAVICS/Hummingbird has lots of WPS services
### Let's keep it simple with 'ncdump'  
* Print info on WPS inputs needed

In [13]:
# ncdump
proc_name = 'ncdump'
process = wps.describeprocess(proc_name) # get process info
for inputs in process.dataInputs:
    print(inputs.identifier)

dataset
dataset_opendap


#### The only input we need is a dataset (url) or it's OpenDAP link
* A simple way to find a test dataset is to access : https://pavics.ouranos.ca/thredds

* Note - PAVICS also has a catalogue WPS but we will see that in other examples later

In [14]:
# Exampe netcdf url to NRCAN daily - tasmin 2013
nc_url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/nrcan/nrcan_canada_daily/tasmin/nrcan_canada_daily_tasmin_2013.nc'
print(nc_url)

https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/nrcan/nrcan_canada_daily/tasmin/nrcan_canada_daily_tasmin_2013.nc


#### Create WPS input - Python list

In [16]:
myinputs = []
myinputs.append(('dataset_opendap',nc_url)) # inputs : use opendap link of a single netcdf file from catalogue search to erun ncdump
print(myinputs)

[('dataset_opendap', 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/nrcan/nrcan_canada_daily/tasmin/nrcan_canada_daily_tasmin_2013.nc')]


#### Execute the WPS

In [17]:
print(proc_name)
execute = wps.execute(proc_name, myinputs)
execute

# Note- execute.processOutputs[-1].reference should provide url link directly but does not here?
# execute.processOutputs[-1].reference   

# Work around using statuslocation xml as ElementTree (ET) package 
o = requests.get(execute.statusLocation)
tree = ET.fromstring(o.content)
for child in tree:
    if 'ProcessOutputs' in child.tag:
        for c in child:
            if 'Output' in c.tag:
                for cc in c: 
                    if 'Reference' in cc.tag:
                        r = requests.get(cc.get('{http://www.w3.org/1999/xlink}href'))


ncdump


#### Get result and print to screen


In [9]:
execute.statusLocation
print('\n\nNCdump results:\n\n' + r.text)




NCdump results:

netcdf nrcan_canada_daily_tasmin_2013.nc {
dimensions:
	time = UNLIMITED ; // (365 currently)
	lat = 510 ;
	lon = 1068 ;
	ts = 3 ;
variables:
	float lon(lon) ;
		lon:units = "degrees_east" ;
		lon:long_name = "longitude" ;
		lon:standard_name = "longitude" ;
		lon:axis = "X" ;
		lon:_ChunkSizes = 1068 ;
	float lat(lat) ;
		lat:axis = "Y" ;
		lat:units = "degrees_north" ;
		lat:long_name = "latitude" ;
		lat:standard_name = "latitude" ;
		lat:_ChunkSizes = 510 ;
	short ts(ts) ;
		ts:_FillValue = -32767s ;
		ts:_ChunkSizes = 3 ;
	short time(time) ;
		time:axis = "T" ;
		time:units = "days since 1950-01-01 00:00:00" ;
		time:long_name = "time" ;
		time:standard_name = "time" ;
		time:calendar = "gregorian" ;
		time:_ChunkSizes = 1 ;
	short time_vectors(time, ts) ;
		time_vectors:_ChunkSizes = 1, 3 ;
	float tasmin(time, lat, lon) ;
		tasmin:long_name = "air_temperature" ;
		tasmin:standard_name = "air_temperature" ;
		tasmin:units = "K" ;
		tasmin:_FillValue = 9.96921e+3